In [7]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
import numpy as np
import tensorflow as tf
import os

In [2]:
def get_mnist(train_size, test_size):
    mnist = fetch_mldata('MNIST original')
    X = mnist.data.astype('float64')
    y = mnist.target
    random_state = check_random_state(0)


    x_train, x_test, y_train, y_test = train_test_split(X, y, 
                        train_size=train_size, test_size=test_size, random_state= random_state)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    return x_train, x_test, y_train, y_test

#expected = y_test
#predicted = y_test #Network output

#print("Classification report:")
#print(classification_report(expected, predicted))
#print("Confusion matrix:")
#print(confusion_matrix(expected, predicted))

def get_logs_path(session):
    """ Returns path where log is going to be saved.
    @Input session is the parent folder for the runs
    So e.g. (Session1 -> run_00, run_01, run_02 ),
    (Session2 -> run_00, run_01)"""
    curr_dir = os.getcwd()
    logs_path = curr_dir + '/tensorflow_logs/train_mnist/' + session
    if not os.path.exists(logs_path):
        os.makedirs(logs_path)
    previous_runs = os.listdir(logs_path)
    new_path = logs_path + "/run_{:02d}".format(len(previous_runs))
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        print("Saving log at: ", new_path)
        return new_path
    raise Exception("Path is already existing: ", new_path)

In [3]:
""" Ziel ist eine nn struktur die sich durch tweaken von eins zwei parametern komplett verändern lässt
dabei sollte die depth variable sein, so wie wie viele hidden units ich habe
wie die variablen aufgesetzt werden sollte auch per input regulierbar sein
Maybe with dropout etc."""

def forward(input_samples, num_hidden=2, output_units=[20,10,10,10]):
    """ this is the whole network structure, returns the last layer after computing
    every other layer"""
    
    #assert that inputs are vaguely corretct:
    assert len(output_units) == num_hidden + 2,\
    "output_units length: {}, while only {} num_hidden".format(len(output_units),  num_hidden)
    
    #reshape input data
    input_layer = input_samples
    
    #input layer
    first_layer = tf.layers.dense(inputs=input_layer, units=output_units[0], name="Input_layer")
    
    layers = {
        "l0": first_layer 
    }
    
    #some deep layers num_hidden iterations, just shifted one forward
    for i in range(1,num_hidden+1):
        prev_layer = layers["l"+str(i-1)]
        layers["l"+str(i)] = tf.layers.dense(inputs=prev_layer, 
                                             units=output_units[i], name="Hidden_layer_"+str(i))
        
    
    #output layer
    out_layer = tf.layers.dense(inputs=layers["l"+str(num_hidden)], units=output_units[-1], name="Output_Layer")
    return out_layer

In [10]:
def train(x_train, y_train, batch_size, input_size, num_epochs, writer):
    """ trains the network defined in forward """
    
    x_ph = tf.placeholder(tf.float32, [batch_size, input_size])
    y_ph = tf.placeholder(tf.int32, [batch_size])
    
    print("x placeholder: ", x_ph)
    prediction = forward(x_ph)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels= y_ph, logits=prediction)
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss)
    
    tf.summary.scalar("loss",loss)
    summ = tf.summary.merge_all()
    
    batches_per_epoch = x_train.shape[0] // batch_size
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(num_epochs):
            for batch in range(batches_per_epoch):
                x_batch = x_train[batch * batch_size : (batch+1) * batch_size]
                y_batch = y_train[batch * batch_size : (batch+1) * batch_size]
                
                _, cur_loss, cur_summ = sess.run([train_op, loss, summ], feed_dict={x_ph: x_batch, y_ph: y_batch})
                writer.add_summary(cur_summ, epoch*batches_per_epoch + batch)
                print(cur_loss)

In [12]:
def run_model():
    x_train, y_train, x_test, y_test = get_mnist(1000,300)
    logs_path = get_logs_path("Session_00")
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    train(x_train=x_train, y_train=y_train, batch_size=100, input_size=784, num_epochs=5, writer=writer)

In [13]:
run_model()

Saving log at:  E:\Programmieren\Python\learnPython\trial_and_error\Projekt_Deep_arch/tensorflow_logs/train_mnist/Session_00/run_02
x placeholder:  Tensor("Placeholder_2:0", shape=(100, 784), dtype=float32)


ValueError: Variable Input_layer/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1718, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3392, in create_op
    op_def=op_def)
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)
